# Simple Example 

In [ ]:
from common.ETLBase import PipelineBuilder, PipeConfigBuilder

In [ ]:
config = PipeConfigBuilder()
pipe = PipelineBuilder(config)

In [ ]:
config.setups(a=1,b=2)

In [ ]:
config.view()

In [ ]:
config.view(summary=False)

In [ ]:
@pipe._func_
def plus_a_b(a=1,b=2):
    return a+b
#pipe._plus_a_b = plus_a_b

In [ ]:
pipe.setup_connection('c = plus_a_b(a=a,b=b)')
pipe.setup_connection('d = plus_a_b(a=a,b=c)')
pipe.setup_connection('e = plus_a_b(d,d)')
pipe.setup_connection('f = plus_a_b(a=b,b=d)')

pipe.view(summary=False)

In [ ]:
pipe.f.get()

# Visualization of the Data Pipeline in an Experiment 

In [ ]:
from experiments.ex3.connect_pipeline import pipe

In [ ]:
pipe.view(summary=False)

In [ ]:
pipe.view_dependency('split_data',summary=False)

# Obtain values of a temporary result of the pipeline 

In [ ]:
pipe.test_dataset.get(verbose=True)

# Example of making modification to the pipeline 

## obtain temporary result 

In [ ]:
pipe.df_y.get()

# define new function and connect in the end 

In [ ]:
from experiments.ex3.connect_pipeline import pipe
@pipe._func_
def get_length(tensor_dataset):
    return len(tensor_dataset)

pipe.setup_connection('size = get_length(train_dataset)')

In [ ]:
pipe.size.get()

In [ ]:
pipe.view(summary=False)

# viewing data content, size, and type 

In [ ]:
from experiments.ex3.connect_pipeline import config, pipe

In [ ]:
pipe.view(summary=False, option = 'all')

# modify a function

In [1]:
from experiments.ex3.connect_pipeline import config, pipe
# pipe.view(summary=False)

In [ ]:
@pipe.replace_function('ProcessY.process')
def ProcessY_process(Y_train, Y_test, columns):
    return len(Y_train), len(Y_test), len(Y_train), len(Y_test), len(Y_train), len(Y_test)

In [ ]:
pipe.test_label_0.get()

In [2]:
@pipe._rep_func_('build_TensorDataset')
def build_TensorDataset(x_dense, x_sparse, objmean, tscnt, label_0):
    return len(x_dense)

In [3]:
pipe.test_dataset.get()

100%|██████████| 50/50 [00:06<00:00,  7.24it/s]


Processing x_train
Split Sparse and Dense Parts and Apply np.float64 and np.int64, respectively
apply Log(1+x) transformation to the dense features
apply MinMaxScale((0,1)) to the dense features
Processing x_test
Split Sparse and Dense Parts and Apply np.float64 and np.int64, respectively
apply Log(1+x) transformation to the dense features
apply MinMaxScale((0,1)) to the dense features
Processing x_train
Split Sparse and Dense Parts and Apply np.float64 and np.int64, respectively
apply Log(1+x) transformation to the dense features
apply MinMaxScale((0,1)) to the dense features
Processing x_test
Split Sparse and Dense Parts and Apply np.float64 and np.int64, respectively
apply Log(1+x) transformation to the dense features
apply MinMaxScale((0,1)) to the dense features


100

In [ ]:
pipe.train_dataset.get()